<a href="https://colab.research.google.com/github/kumarsonam/Colab/blob/main/llama3/Llama3HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Thu May  2 13:27:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers datasets evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
!pip install trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-ma

In [4]:
!pip install sentencepiece

In [5]:
!pip install accelerate -U

In [6]:
!pip install transformers[torch]

In [7]:
from datasets import load_dataset

In [8]:
dataset = load_dataset("databricks/databricks-dolly-15k")

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [9]:
dataset_split = dataset['train'].train_test_split(test_size=.01)

In [10]:
print(dataset_split)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 14860
    })
    test: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 151
    })
})


In [11]:
# Logits: [sequence_length, vocab_size]
# Probability distribution of where each location in the sequence length is predicting the next token
# over all possible tokens in the models vocab.

# Softmax.

import torch
import evaluate

def preprocess_logits_for_metrics(logits, labels):
  if isinstance(logits, tuple):
    logits = logits[0]
  return logits.argmax(dim=-1)

metric = evaluate.load('accuracy')

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  # Shift the labels and predictions
  labels = labels[:, 1:].reshape(-1)
  preds = preds[:, :-1].reshape(-1)

  mask = labels != -100 # boolean mask

  # apply the mask to filter out ignored labels and corresponding predictions
  filtered_labels = labels[mask]
  filtered_preds = preds[mask]

  return metric.compute(predictions=filtered_preds, references=filtered_labels)

In [12]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T", torch_dtype = torch.bfloat16, device_map='auto')

# Processing a raw example
def formatting_prompts_func(example):
  output_texts = []
  for i in range(len(example['instruction'])):
    text = f"### Input: {example['context'][i] + ' ' + example['instruction'][i]}\n ### Output: {example['response'][i]}"
    output_texts.append(text)
  return output_texts

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [13]:
# Add context to response template
response_template_with_context = "### Input: Dummy input\n ### Output:"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)
print(response_template_ids)
response_template_ids = response_template_ids[-1*len(tokenizer.encode(" ### Output:", add_special_tokens=False)):]
print(response_template_ids)
collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

[835, 10567, 29901, 360, 11770, 1881, 13, 835, 10604, 29901]
[13, 835, 10604, 29901]


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    gradient_accumulation_steps=32,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=10,
    weight_decay=0.01,
    evaluation_strategy='steps',
    eval_steps=10, # evaluate every 10 steps
    logging_steps=1,
    gradient_checkpointing=True, # recomputes forward pass activations in backward pass to save memory
    save_steps=500 # checkpoint every 500 steps
)

trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=dataset_split['train'],
    eval_dataset=dataset_split['test'],
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    max_seq_length=2048 # Truncate sequences greater than 2048 tokens. Increasing = more memory usage
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/14860 [00:00<?, ? examples/s]

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,1.442700,1.537005,0.591516
20,1.354400,1.480480,0.594344
30,1.437100,1.480008,0.592435
40,0.998500,1.470560,0.593425
50,1.410200,1.476026,0.593779
60,1.287200,1.469808,0.593920
70,1.530300,1.464699,0.593142
80,1.465600,1.468045,0.592718
90,1.333500,1.473626,0.592577
100,1.512100,1.471248,0.591092


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key `[13, 835, 10604, 29901]` in the following instance: <s> ### Input: The ANT catalog (or TAO catalog) is a classified product catalog by the U.S. National Security Agency (NSA) of which the version written in 2008–2009 was published by German news magazine Der Spiegel in December 2013. Forty-nine catalog pages with pictures, diagrams and descriptions of espionage devices and spying software were published. The items are available to th

Step,Training Loss,Validation Loss,Accuracy
10,1.442700,1.537005,0.591516
20,1.354400,1.480480,0.594344
30,1.437100,1.480008,0.592435
40,0.998500,1.470560,0.593425
50,1.410200,1.476026,0.593779
60,1.287200,1.469808,0.593920
70,1.530300,1.464699,0.593142
80,1.465600,1.468045,0.592718
90,1.333500,1.473626,0.592577
100,1.512100,1.471248,0.591092


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
